In [1]:
import pandas as pd


pivot_file = r"..\result\pivot_df.xlsx"

pivot_TABLE = pd.read_excel(pivot_file)
pivot_TABLE

,ENGY_ESNCNO,UNIT_CD,SEQ_NO,MGM_BLD_PK,E사용량 (kWh)_전기 (KWH),E사용량 (kWh)_가스 (KWH),E사용량 (kWh)_지역냉난방 (KWH),USE_QTY_11,USE_QTY_12,USE_QTY_13
0,4,3,5334976,44810-100214543,2.132705e+06,1.272160e+05,415847.427460,NaN,NaN,491.16
1,29,6,774699,41370-100175872,6.545150e+05,0.000000e+00,194127.570240,NaN,NaN,250280.00
2,46,3,2473583,30200-100177670,2.063007e+06,1.430016e+04,498261.633811,NaN,NaN,298.66
3,46,6,2473583,30200-100177670,2.063007e+06,1.430016e+04,498261.633811,NaN,NaN,588500.00
4,59,6,6284469,30230-10947,6.254510e+05,9.668293e+04,887896.396000,NaN,NaN,1048700.00
...,...,...,...,...,...,...,...,...,...,...
9649,6104511013,8,4443068,28177-17939,2.161598e+06,1.971172e+06,0.000000,NaN,3.609187e+04,NaN
9650,6104511324,8,603578,41192-100192071,4.993679e+05,6.170120e+05,0.000000,NaN,1.972469e+04,NaN
9651,12769626346,8,4091378,31200-7813,1.330900e+05,2.208885e+05,0.000000,NaN,1.109863e+06,NaN
9652,15900262179,8,4101590,31140-9248,2.188815e+06,3.338372e+04,0.000000,NaN,2.014162e+06,NaN


In [2]:
pivot_TABLE['MGM_BLD_PK'].value_counts()

MGM_BLD_PK
11200-26004        544
41287-231300        81
11200-6758          68
11200-100199932     67
41285-132553        63
                  ... 
51150-13241          1
51110-5939           1
51230-2203           1
51230-2207           1
51230-100174426      1
Name: count, Length: 1570, dtype: int64

In [3]:
pivot_TABLE['SEQ_NO'].value_counts()

SEQ_NO
3273039    544
6284466     81
3721187     68
6355104     67
521058      63
          ... 
41385        1
181732       1
183536       1
183527       1
184985       1
Name: count, Length: 1570, dtype: int64

In [5]:

# 1. SEQ_NO별로 MGM_BLD_PK의 고유값 개수를 확인
seq_group = pivot_TABLE.groupby('SEQ_NO')['MGM_BLD_PK'].nunique().reset_index(name='unique_mgm_count')
# unique_mgm_count가 1이 아닌 경우가 있다면, 같은 SEQ_NO에 대해 다른 MGM_BLD_PK가 존재함
inconsistent_seq = seq_group[seq_group['unique_mgm_count'] != 1]

if inconsistent_seq.empty:
    print("모든 SEQ_NO에 대해, 동일한 MGM_BLD_PK가 할당되어 있습니다.")
else:
    print("일부 SEQ_NO에 대해 서로 다른 MGM_BLD_PK가 할당되어 있습니다:")
    print(inconsistent_seq)

# 2. 반대로 MGM_BLD_PK별로 SEQ_NO의 고유값 개수를 확인해봅니다.
mgm_group = pivot_TABLE.groupby('MGM_BLD_PK')['SEQ_NO'].nunique().reset_index(name='unique_seq_count')
inconsistent_mgm = mgm_group[mgm_group['unique_seq_count'] != 1]

if inconsistent_mgm.empty:
    print("모든 MGM_BLD_PK에 대해, 동일한 SEQ_NO가 할당되어 있습니다.")
else:
    print("일부 MGM_BLD_PK에 대해 서로 다른 SEQ_NO가 할당되어 있습니다:")
    print(inconsistent_mgm)


모든 SEQ_NO에 대해, 동일한 MGM_BLD_PK가 할당되어 있습니다.
모든 MGM_BLD_PK에 대해, 동일한 SEQ_NO가 할당되어 있습니다.


In [34]:


# USE_QTY 합계 계산하여 새로운 컬럼으로 추가
# nan만 있는 경우 nan으로 유지하기 위한 함수 정의
def sum_with_nan_check(series):
    if series.isna().all():
        return float('nan')
    return series.sum()

# # 각 SEQ_NO 그룹별로 합계 계산 (nan만 있는 경우 nan 유지)
# pivot_TABLE['USE_QTY_11_SUM'] = pivot_TABLE.groupby('SEQ_NO')['USE_QTY_11'].transform(sum_with_nan_check)
# pivot_TABLE['USE_QTY_12_SUM'] = pivot_TABLE.groupby('SEQ_NO')['USE_QTY_12'].transform(sum_with_nan_check)
# pivot_TABLE['USE_QTY_13_SUM'] = pivot_TABLE.groupby('SEQ_NO')['USE_QTY_13'].transform(sum_with_nan_check)

# 각 UNIT_CD + SEQ_NO 그룹별로 합계 계산 (nan만 있는 경우 nan 유지)
pivot_TABLE['USE_QTY_11_UNIT_SUM'] = pivot_TABLE.groupby(['UNIT_CD', 'SEQ_NO'])['USE_QTY_11'].transform(sum_with_nan_check)
pivot_TABLE['USE_QTY_12_UNIT_SUM'] = pivot_TABLE.groupby(['UNIT_CD', 'SEQ_NO'])['USE_QTY_12'].transform(sum_with_nan_check)
pivot_TABLE['USE_QTY_13_UNIT_SUM'] = pivot_TABLE.groupby(['UNIT_CD', 'SEQ_NO'])['USE_QTY_13'].transform(sum_with_nan_check)

# 결과 확인
print("\n결과 데이터:")
pivot_TABLE


결과 데이터:


,ENGY_ESNCNO,UNIT_CD,SEQ_NO,MGM_BLD_PK,E사용량 (kWh)_전기 (KWH),E사용량 (kWh)_가스 (KWH),E사용량 (kWh)_지역냉난방 (KWH),USE_QTY_11,USE_QTY_12,USE_QTY_13,USE_QTY_11_SUM,USE_QTY_12_SUM,USE_QTY_13_SUM,USE_QTY_11_UNIT_SUM,USE_QTY_12_UNIT_SUM,USE_QTY_13_UNIT_SUM
0,4,3,5334976,44810-100214543,2.132705e+06,1.272160e+05,415847.427460,NaN,NaN,491.16,32249.0,1.102934e+06,491.16,NaN,NaN,491.16
1,29,6,774699,41370-100175872,6.545150e+05,0.000000e+00,194127.570240,NaN,NaN,250280.00,657512.0,NaN,250280.00,NaN,NaN,250280.00
2,46,3,2473583,30200-100177670,2.063007e+06,1.430016e+04,498261.633811,NaN,NaN,298.66,2565837.0,5.531000e+04,588798.66,NaN,NaN,298.66
3,46,6,2473583,30200-100177670,2.063007e+06,1.430016e+04,498261.633811,NaN,NaN,588500.00,2565837.0,5.531000e+04,588798.66,NaN,NaN,588500.00
4,59,6,6284469,30230-10947,6.254510e+05,9.668293e+04,887896.396000,NaN,NaN,1048700.00,632666.0,3.526831e+05,1048700.00,NaN,NaN,1048700.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,6104511013,8,4443068,28177-17939,2.161598e+06,1.971172e+06,0.000000,NaN,3.609187e+04,NaN,2226834.0,9.352199e+06,NaN,NaN,9.352199e+06,NaN
9650,6104511324,8,603578,41192-100192071,4.993679e+05,6.170120e+05,0.000000,NaN,1.972469e+04,NaN,670376.0,2.115874e+06,NaN,NaN,2.115874e+06,NaN
9651,12769626346,8,4091378,31200-7813,1.330900e+05,2.208885e+05,0.000000,NaN,1.109863e+06,NaN,188258.0,1.109863e+06,NaN,NaN,1.109863e+06,NaN
9652,15900262179,8,4101590,31140-9248,2.188815e+06,3.338372e+04,0.000000,NaN,2.014162e+06,NaN,18884788.0,2.536702e+06,NaN,NaN,2.536702e+06,NaN


In [35]:
# dedup_df를 "dedup_df.xlsx" 파일로 저장 (인덱스 없이)
pivot_TABLE.to_excel(r"../result/pivot_df_ver3.xlsx", index=False)
pivot_TABLE

,ENGY_ESNCNO,UNIT_CD,SEQ_NO,MGM_BLD_PK,E사용량 (kWh)_전기 (KWH),E사용량 (kWh)_가스 (KWH),E사용량 (kWh)_지역냉난방 (KWH),USE_QTY_11,USE_QTY_12,USE_QTY_13,USE_QTY_11_SUM,USE_QTY_12_SUM,USE_QTY_13_SUM,USE_QTY_11_UNIT_SUM,USE_QTY_12_UNIT_SUM,USE_QTY_13_UNIT_SUM
0,4,3,5334976,44810-100214543,2.132705e+06,1.272160e+05,415847.427460,NaN,NaN,491.16,32249.0,1.102934e+06,491.16,NaN,NaN,491.16
1,29,6,774699,41370-100175872,6.545150e+05,0.000000e+00,194127.570240,NaN,NaN,250280.00,657512.0,NaN,250280.00,NaN,NaN,250280.00
2,46,3,2473583,30200-100177670,2.063007e+06,1.430016e+04,498261.633811,NaN,NaN,298.66,2565837.0,5.531000e+04,588798.66,NaN,NaN,298.66
3,46,6,2473583,30200-100177670,2.063007e+06,1.430016e+04,498261.633811,NaN,NaN,588500.00,2565837.0,5.531000e+04,588798.66,NaN,NaN,588500.00
4,59,6,6284469,30230-10947,6.254510e+05,9.668293e+04,887896.396000,NaN,NaN,1048700.00,632666.0,3.526831e+05,1048700.00,NaN,NaN,1048700.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,6104511013,8,4443068,28177-17939,2.161598e+06,1.971172e+06,0.000000,NaN,3.609187e+04,NaN,2226834.0,9.352199e+06,NaN,NaN,9.352199e+06,NaN
9650,6104511324,8,603578,41192-100192071,4.993679e+05,6.170120e+05,0.000000,NaN,1.972469e+04,NaN,670376.0,2.115874e+06,NaN,NaN,2.115874e+06,NaN
9651,12769626346,8,4091378,31200-7813,1.330900e+05,2.208885e+05,0.000000,NaN,1.109863e+06,NaN,188258.0,1.109863e+06,NaN,NaN,1.109863e+06,NaN
9652,15900262179,8,4101590,31140-9248,2.188815e+06,3.338372e+04,0.000000,NaN,2.014162e+06,NaN,18884788.0,2.536702e+06,NaN,NaN,2.536702e+06,NaN


In [36]:
pivot_df = pivot_TABLE[['ENGY_ESNCNO','SEQ_NO', 'MGM_BLD_PK', 'UNIT_CD', 'USE_QTY_11_UNIT_SUM', 'USE_QTY_12_UNIT_SUM', 'USE_QTY_13_UNIT_SUM', 'E사용량 (kWh)_전기 (KWH)', 'E사용량 (kWh)_가스 (KWH)','E사용량 (kWh)_지역냉난방 (KWH)']]
pivot_df

,ENGY_ESNCNO,SEQ_NO,MGM_BLD_PK,UNIT_CD,USE_QTY_11_UNIT_SUM,USE_QTY_12_UNIT_SUM,USE_QTY_13_UNIT_SUM,E사용량 (kWh)_전기 (KWH),E사용량 (kWh)_가스 (KWH),E사용량 (kWh)_지역냉난방 (KWH)
0,4,5334976,44810-100214543,3,NaN,NaN,491.16,2.132705e+06,1.272160e+05,415847.427460
1,29,774699,41370-100175872,6,NaN,NaN,250280.00,6.545150e+05,0.000000e+00,194127.570240
2,46,2473583,30200-100177670,3,NaN,NaN,298.66,2.063007e+06,1.430016e+04,498261.633811
3,46,2473583,30200-100177670,6,NaN,NaN,588500.00,2.063007e+06,1.430016e+04,498261.633811
4,59,6284469,30230-10947,6,NaN,NaN,1048700.00,6.254510e+05,9.668293e+04,887896.396000
...,...,...,...,...,...,...,...,...,...,...
9649,6104511013,4443068,28177-17939,8,NaN,9.352199e+06,NaN,2.161598e+06,1.971172e+06,0.000000
9650,6104511324,603578,41192-100192071,8,NaN,2.115874e+06,NaN,4.993679e+05,6.170120e+05,0.000000
9651,12769626346,4091378,31200-7813,8,NaN,1.109863e+06,NaN,1.330900e+05,2.208885e+05,0.000000
9652,15900262179,4101590,31140-9248,8,NaN,2.536702e+06,NaN,2.188815e+06,3.338372e+04,0.000000


In [19]:
# ENGY_ESNCNO 컬럼에서 중복된 값(두 개 이상 있는 값들) 필터링
duplicates = pivot_df[pivot_df.duplicated(subset='ENGY_ESNCNO', keep=False)]

# 중복된 ENGY_ESNCNO의 고유값 개수 출력
dup_unique_count = duplicates['ENGY_ESNCNO'].nunique()
print("중복된 ENGY_ESNCNO 고유값 개수:", dup_unique_count)

# 중복된 행 전체 출력
print("중복된 행들:")
print(duplicates)


중복된 ENGY_ESNCNO 고유값 개수: 1
중복된 행들:
   ENGY_ESNCNO   SEQ_NO       MGM_BLD_PK  UNIT_CD  USE_QTY_11_SUM  \
2           46  2473583  30200-100177670        3       2565837.0   
3           46  2473583  30200-100177670        6       2565837.0   

   USE_QTY_12_SUM  USE_QTY_13_SUM  E사용량 (kWh)_전기 (KWH)  E사용량 (kWh)_가스 (KWH)  \
2      55309.9963       588798.66         2.063007e+06         14300.160034   
3      55309.9963       588798.66         2.063007e+06         14300.160034   

   E사용량 (kWh)_지역냉난방 (KWH)  
2           498261.633811  
3           498261.633811  


In [9]:
# "ENGY_ESNCNO"를 제외한 나머지 컬럼 목록 생성
cols_to_check = [col for col in pivot_df.columns if col != 'ENGY_ESNCNO']

# 해당 컬럼들을 기준으로 중복된 행들을 모두 찾기 (keep=False: 모든 중복 행 표시)
duplicate_rows = pivot_df[pivot_df.duplicated(subset=cols_to_check, keep=False)]

# 결과 출력
print("ENGY_ESNCNO 컬럼 제외 중복 행:")
print(duplicate_rows)


ENGY_ESNCNO 컬럼 제외 중복 행:
      ENGY_ESNCNO   SEQ_NO       MGM_BLD_PK  UNIT_CD  USE_QTY_11_SUM  \
14          10384  2001491       41410-8154        3        350814.0   
17          10395   483933       41410-5978        3        481333.0   
18          10400   483933       41410-5978        3        481333.0   
23          10774   604538  41192-100213375        3        401257.0   
37          11198   604538  41192-100213375        3        401257.0   
...           ...      ...              ...      ...             ...   
9648   6104457001   603446     41196-226965        8        266841.0   
9649   6104511013  4443068      28177-17939        8       2226834.0   
9650   6104511324   603578  41192-100192071        8        670376.0   
9652  15900262179  4101590       31140-9248        8      18884788.0   
9653  16136301854  4103049       31140-9259        8             0.0   

      USE_QTY_12_SUM  USE_QTY_13_SUM  E사용량 (kWh)_전기 (KWH)  \
14      0.000000e+00           98.86         3.330

In [28]:
# 모든 열을 기준으로 중복 제거
# 모든 컬럼을 사용하여 중복 제거 (ENGY_ESNCNO 포함)
cols_to_check = pivot_df.columns.tolist()

# 지정한 컬럼들을 기준으로 중복 제거 (각 그룹에서 첫 번째 행만 남김)
dedup_df = pivot_df.drop_duplicates(subset=cols_to_check, keep='first')

# 남은 건수 출력
print("모든 열 기준 중복 제거 후 남은 건수:", len(dedup_df))


모든 열 기준 중복 제거 후 남은 건수: 9654


In [38]:
# 'ENGY_ESNCNO'를 제외한 나머지 컬럼 목록 생성

cols_to_check = [col for col in pivot_df.columns if col != 'ENGY_ESNCNO']


# 지정한 컬럼들을 기준으로 중복 제거 (각 그룹에서 첫 번째 행만 남김)
dedup_df = pivot_df.drop_duplicates(subset=cols_to_check, keep='first')

# 남은 건수 출력
print("중복 제거 후 남은 건수:", len(dedup_df))


중복 제거 후 남은 건수: 2494


In [40]:
dedup_df

,ENGY_ESNCNO,SEQ_NO,MGM_BLD_PK,UNIT_CD,USE_QTY_11_UNIT_SUM,USE_QTY_12_UNIT_SUM,USE_QTY_13_UNIT_SUM,E사용량 (kWh)_전기 (KWH),E사용량 (kWh)_가스 (KWH),E사용량 (kWh)_지역냉난방 (KWH)
0,4,5334976,44810-100214543,3,NaN,NaN,491.16,2.132705e+06,127215.991200,415847.427460
1,29,774699,41370-100175872,6,NaN,NaN,250280.00,6.545150e+05,0.000000,194127.570240
2,46,2473583,30200-100177670,3,NaN,NaN,298.66,2.063007e+06,14300.160034,498261.633811
3,46,2473583,30200-100177670,6,NaN,NaN,588500.00,2.063007e+06,14300.160034,498261.633811
4,59,6284469,30230-10947,6,NaN,NaN,1048700.00,6.254510e+05,96682.925000,887896.396000
...,...,...,...,...,...,...,...,...,...,...
9626,6104149433,5774831,41210-9379,8,NaN,3.118695e+05,NaN,4.562660e+05,50459.847750,0.000000
9630,6104220353,542780,41210-4723,8,NaN,1.510037e+06,NaN,6.859260e+05,388861.385000,0.000000
9643,6104381138,854471,41220-100235920,8,NaN,9.848801e+02,NaN,3.046760e+05,64828.071810,0.000000
9645,6104447237,56929,41410-5897,8,NaN,2.805949e+05,NaN,2.525450e+05,76578.050815,14638.758874


In [41]:
# 1. 전기 (ENGY_type 11) 비교
mask_elec = dedup_df['USE_QTY_11_SUM'] == dedup_df['E사용량 (kWh)_전기 (KWH)']
elec_matching_count = mask_elec.sum()
elec_non_matching_count = len(dedup_df) - elec_matching_count

# 2. 가스 (ENGY_type 12) 비교
mask_gas = dedup_df['USE_QTY_12_SUM'] == dedup_df['E사용량 (kWh)_가스 (KWH)']
gas_matching_count = mask_gas.sum()
gas_non_matching_count = len(dedup_df) - gas_matching_count

# 3. 지역냉난방 (ENGY_type 13) 비교
mask_local = dedup_df['USE_QTY_13_SUM'] == dedup_df['E사용량 (kWh)_지역냉난방 (KWH)']
local_matching_count = mask_local.sum()
local_non_matching_count = len(dedup_df) - local_matching_count

# 결과 출력
print("전기 (ENGY_type 11) 일치 여부:")
print(" - 일치하는 행의 개수:", elec_matching_count)
print(" - 일치하지 않는 행의 개수:", elec_non_matching_count)
print()
print("가스 (ENGY_type 12) 일치 여부:")
print(" - 일치하는 행의 개수:", gas_matching_count)
print(" - 일치하지 않는 행의 개수:", gas_non_matching_count)
print()
print("지역냉난방 (ENGY_type 13) 일치 여부:")
print(" - 일치하는 행의 개수:", local_matching_count)
print(" - 일치하지 않는 행의 개수:", local_non_matching_count)

# 일치하는 행 출력
print("\n전기 (ENGY_type 11) 일치하는 행:")
print(dedup_df[mask_elec])

print("\n가스 (ENGY_type 12) 일치하는 행:")
print(dedup_df[mask_gas])

print("\n지역냉난방 (ENGY_type 13) 일치하는 행:")
print(dedup_df[mask_local])


KeyError: 'USE_QTY_11_SUM'

In [45]:
# 1. 전기 (ENGY_type 11) 비교 - NaN 값 제외
mask_elec_valid = dedup_df['USE_QTY_11_UNIT_SUM'].notna()
valid_elec_df = dedup_df[mask_elec_valid]
mask_elec = valid_elec_df['USE_QTY_11_UNIT_SUM'] == valid_elec_df['E사용량 (kWh)_전기 (KWH)']
elec_matching_count = mask_elec.sum()
elec_non_matching_count = len(valid_elec_df) - elec_matching_count

# 2. 가스 (ENGY_type 12) 비교 - NaN 값 제외
mask_gas_valid = dedup_df['USE_QTY_12_UNIT_SUM'].notna()
valid_gas_df = dedup_df[mask_gas_valid]
mask_gas = valid_gas_df['USE_QTY_12_UNIT_SUM'] == valid_gas_df['E사용량 (kWh)_가스 (KWH)']
gas_matching_count = mask_gas.sum()
gas_non_matching_count = len(valid_gas_df) - gas_matching_count

# 3. 지역냉난방 (ENGY_type 13) 비교 - NaN 값 제외
mask_local_valid = dedup_df['USE_QTY_13_UNIT_SUM'].notna()
valid_local_df = dedup_df[mask_local_valid]
mask_local = valid_local_df['USE_QTY_13_UNIT_SUM'] == valid_local_df['E사용량 (kWh)_지역냉난방 (KWH)']
local_matching_count = mask_local.sum()
local_non_matching_count = len(valid_local_df) - local_matching_count

# 결과 출력
print("전기 (ENGY_type 11) 일치 여부 (NaN 제외):")
print(" - 유효한 행의 개수:", len(valid_elec_df))
print(" - 일치하는 행의 개수:", elec_matching_count)
print(" - 일치하지 않는 행의 개수:", elec_non_matching_count)
print()
print("가스 (ENGY_type 12) 일치 여부 (NaN 제외):")
print(" - 유효한 행의 개수:", len(valid_gas_df))
print(" - 일치하는 행의 개수:", gas_matching_count)
print(" - 일치하지 않는 행의 개수:", gas_non_matching_count)
print()
print("지역냉난방 (ENGY_type 13) 일치 여부 (NaN 제외):")
print(" - 유효한 행의 개수:", len(valid_local_df))
print(" - 일치하는 행의 개수:", local_matching_count)
print(" - 일치하지 않는 행의 개수:", local_non_matching_count)

# 일치하는 행 출력
print("\n전기 (ENGY_type 11) 일치하는 행:")
print(valid_elec_df[mask_elec])

print("\n가스 (ENGY_type 12) 일치하는 행:")
print(valid_gas_df[mask_gas])

print("\n지역냉난방 (ENGY_type 13) 일치하는 행:")
print(valid_local_df[mask_local])


전기 (ENGY_type 11) 일치 여부 (NaN 제외):
 - 유효한 행의 개수: 1370
 - 일치하는 행의 개수: 2
 - 일치하지 않는 행의 개수: 1368

가스 (ENGY_type 12) 일치 여부 (NaN 제외):
 - 유효한 행의 개수: 981
 - 일치하는 행의 개수: 16
 - 일치하지 않는 행의 개수: 965

지역냉난방 (ENGY_type 13) 일치 여부 (NaN 제외):
 - 유효한 행의 개수: 143
 - 일치하는 행의 개수: 1
 - 일치하지 않는 행의 개수: 142

전기 (ENGY_type 11) 일치하는 행:
      ENGY_ESNCNO   SEQ_NO    MGM_BLD_PK  UNIT_CD  USE_QTY_11_UNIT_SUM  \
1800    141475892  3393003    11680-7409        1             615859.0   
3627    227222902  5775121  41135-122837        1             269340.0   

      USE_QTY_12_UNIT_SUM  USE_QTY_13_UNIT_SUM  E사용량 (kWh)_전기 (KWH)  \
1800                  NaN                  NaN             615859.0   
3627                  NaN                  NaN             269340.0   

      E사용량 (kWh)_가스 (KWH)  E사용량 (kWh)_지역냉난방 (KWH)  
1800           458826.925                     0.0  
3627             5287.450                106391.8  

가스 (ENGY_type 12) 일치하는 행:
      ENGY_ESNCNO          SEQ_NO       MGM_BLD_PK  UNIT_CD  \
112      

In [43]:
# dedup_df를 "dedup_df.xlsx" 파일로 저장 (인덱스 없이)
dedup_df.to_excel(r"../result/dedup_df_ver2.xlsx", index=False)


In [44]:
dedup_df['UNIT_CD'].value_counts()


UNIT_CD
1    1370
8     979
3     122
4      18
6       3
2       2
Name: count, dtype: int64